<a href="https://colab.research.google.com/github/Aithu-Snehith/Neural-networks-Model-optimisation/blob/master/v1_Alexnet_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [0]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly
! pip install -U tf-nightly-gpu

! pip install tensorflow-model-optimization

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
     |████████████████████████████████| 397.4MB 38kB/s 
     |████████████████████████████████| 3.2MB 40.0MB/s 
     |████████████████████████████████| 501kB 45.8MB/s 
     |████████████████████████████████| 61kB 19.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%load_ext tensorboard
import tensorboard

import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os

W0711 09:46:37.622445 140654419400576 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



In [0]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, GaussianDropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from tensorflow.keras.layers import *
import time

Using TensorFlow backend.
W0711 09:46:42.570250 140654419400576 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.



In [0]:
path = 'drive/path_to_data'
model_name = 'alexnet'

batch_size = 128
num_classes = 7
epochs = 200

# input image dimensions
img_rows, img_cols = 48, 48

In [0]:
# x = np.load(path + 'snehith_fer2013_fdataX.npy')
# y = np.load(path + 'snehith_fer2013_flabels.npy')

# print(x.shape, y.shape)

# x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

# np.save(path + 'x_train.npy', x_train)
# np.save(path + 'x_test.npy', x_test)
# np.save(path + 'y_train.npy', y_train)
# np.save(path + 'y_test.npy', y_test)

In [0]:
x_train = np.load(path + 'x_train.npy')
x_test = np.load(path + 'x_test.npy')
y_train = np.load(path + 'y_train.npy')
y_test = np.load(path + 'y_test.npy')

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(32298, 48, 48, 1) (3589, 48, 48, 1) (32298, 7) (3589, 7)


In [0]:
model = Sequential()
# CONV BLOCK 1
model.add(Conv2D(16, (9, 9), input_shape=(img_rows, img_cols, 1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 2
model.add(Conv2D(32, (7, 7), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 3
model.add(Conv2D(64, (5, 5), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 4
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 5
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# FLATTEN
model.add(Flatten())
# DENSE 1
model.add(Dense(1024, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# DENSE 2
model.add(Dense(1024, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 16)        1296      
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 16)        0         
_________________________________________________________________
gaussian_dropout (GaussianDr (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        25088     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        1

In [0]:
logdir = path + model_name + '/logs'
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
    metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0711 05:10:23.568597 140621474404224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0711 05:10:25.215831 140621474404224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 32298 samples, validate on 3589 samples
Epoch 1/200
32298/32298 [==============================] - 8s 256us/sample - loss: 2.0360 - acc: 0.2493 - val_loss: 1.8162 - val_acc: 0.3006
Epoch 2/200
32298/32298 [==============================] - 4s 119us/sample - loss: 1.6870 - acc: 0.3581 - val_loss: 1.7947 - val_acc: 0.3402
Epoch 3/200
32298/32298 [==============================] - 4s 119us/sample - loss: 1.5608 - acc: 0.4011 - val_loss: 1.8600 - val_acc: 0.3277
Epoch 4/200
32298/32298 [==============================] - 4s 119us/sample - loss: 1.4660 - acc: 0.4334 - val_loss: 1.9765 - val_acc: 0.3235
Epoch 5/200
32298/32298 [==============================] - 4s 120us/sample - loss: 1.3983 - acc: 0.4597 - val_loss: 1.4775 - val_acc: 0.4260
Epoch 6/200
32298/32298 [==============================] - 4s 120us/sample - loss: 1.3423 - acc: 0.4837 - val_loss: 1.4657 - val_acc: 0.4355
Epoch 7/200
32298/32298 [==============================] - 4s 120us/sample - loss: 1.2897 - acc: 0.5051 -

In [0]:
# Backend agnostic way to save/restore models
keras_file = path + model_name + '/raw_'+ model_name + '_model.h5'
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  drive/My Drive/interns/Kpit/alexnet/raw_alexnet_model.h5


In [0]:
from tensorflow_model_optimization.sparsity import keras as sparsity

num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

End step: 50600


In [0]:
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=2000,
                                                   end_step=end_step,
                                                   frequency=100)
}


pr_model = Sequential()
# CONV BLOCK 1
pr_model.add(sparsity.prune_low_magnitude(Conv2D(16, (9, 9), input_shape=(img_rows,img_cols, 1), padding='same', use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 2
pr_model.add(sparsity.prune_low_magnitude(Conv2D(32, (7, 7), padding='same', use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 3
pr_model.add(sparsity.prune_low_magnitude(Conv2D(64, (5, 5), padding='same', use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 4
pr_model.add(sparsity.prune_low_magnitude(Conv2D(128, (3, 3), padding='same', use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 5
pr_model.add(sparsity.prune_low_magnitude(Conv2D(128, (3, 3), padding='same', use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# FLATTEN
pr_model.add(Flatten())
# DENSE 1
pr_model.add(sparsity.prune_low_magnitude(Dense(1024, use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Dropout(0.5))
# DENSE 2
pr_model.add(sparsity.prune_low_magnitude(Dense(1024, use_bias=False), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Dropout(0.5))
pr_model.add(sparsity.prune_low_magnitude(Dense(num_classes, activation='softmax'), **pruning_params))
pr_model.summary()

W0711 05:23:23.241508 140621474404224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:185: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
W0711 05:23:23.694135 140621474404224 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

W0711 05:23:23.881768 140621474404224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_schedule.py:240: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_5 (None, 48, 48, 16)        2594      
_________________________________________________________________
batch_normalization_7 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
activation_7 (Activation)    (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
gaussian_dropout_5 (Gaussian (None, 24, 24, 16)        0         
_________________________________________________________________
prune_low_magnitude_conv2d_6 (None, 24, 24, 32)        50178     
_________________________________________________________________
batch_normalization_8 (Batch (None, 24, 24, 32)       

In [0]:
pr_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

pr_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = pr_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 32298 samples, validate on 3589 samples
Epoch 1/200
32298/32298 [==============================] - 9s 283us/sample - loss: 2.0829 - acc: 0.2170 - val_loss: 1.9738 - val_acc: 0.2524
Epoch 2/200
32298/32298 [==============================] - 5s 159us/sample - loss: 1.8041 - acc: 0.2960 - val_loss: 2.0402 - val_acc: 0.2499
Epoch 3/200
32298/32298 [==============================] - 5s 157us/sample - loss: 1.6333 - acc: 0.3685 - val_loss: 1.5991 - val_acc: 0.3789
Epoch 4/200
32298/32298 [==============================] - 5s 156us/sample - loss: 1.5200 - acc: 0.4112 - val_loss: 1.5216 - val_acc: 0.4079
Epoch 5/200
32298/32298 [==============================] - 5s 157us/sample - loss: 1.4382 - acc: 0.4456 - val_loss: 1.4425 - val_acc: 0.4466
Epoch 6/200
32298/32298 [==============================] - 5s 159us/sample - loss: 1.3709 - acc: 0.4724 - val_loss: 1.5362 - val_acc: 0.4157
Epoch 7/200
32298/32298 [==============================] - 5s 157us/sample - loss: 1.3234 - acc: 0.4922 -

In [0]:
checkpoint_file = path + model_name + '/pruned_'+ model_name + '_checkpoint.h5'
print('Saving pruned model to: ', checkpoint_file)
# saved_model() sets include_optimizer to True by default. Spelling it out here
# to highlight.
tf.keras.models.save_model(pr_model, checkpoint_file, include_optimizer=True)

with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(checkpoint_file)

restored_model.fit(x_train, y_train,
                   batch_size=batch_size,
                   epochs=int(epochs/10),
                   verbose=1,
                   callbacks=callbacks,
                   validation_data=(x_test, y_test))

score = restored_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saving pruned model to:  drive/My Drive/interns/Kpit/alexnet/pruned_alexnet_checkpoint.h5
Train on 32298 samples, validate on 3589 samples
Epoch 1/20
32298/32298 [==============================] - 9s 280us/sample - loss: 0.9544 - acc: 0.6407 - val_loss: 1.0187 - val_acc: 0.6180
Epoch 2/20
32298/32298 [==============================] - 5s 160us/sample - loss: 0.9591 - acc: 0.6383 - val_loss: 1.0645 - val_acc: 0.6043
Epoch 3/20
32298/32298 [==============================] - 5s 158us/sample - loss: 0.9607 - acc: 0.6361 - val_loss: 1.0318 - val_acc: 0.6149
Epoch 4/20
32298/32298 [==============================] - 5s 157us/sample - loss: 0.9567 - acc: 0.6387 - val_loss: 1.0305 - val_acc: 0.6147
Epoch 5/20
32298/32298 [==============================] - 5s 157us/sample - loss: 0.9614 - acc: 0.6379 - val_loss: 1.0201 - val_acc: 0.6199
Epoch 6/20
32298/32298 [==============================] - 5s 157us/sample - loss: 0.9571 - acc: 0.6385 - val_loss: 1.0180 - val_acc: 0.6119
Epoch 7/20
32298/3229

In [0]:
final_model = sparsity.strip_pruning(pr_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 48, 48, 16)        1296      
_________________________________________________________________
batch_normalization_7 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
activation_7 (Activation)    (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
gaussian_dropout_5 (Gaussian (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 32)        25088     
_________________________________________________________________
batch_normalization_8 (Batch (None, 24, 24, 32)       

In [0]:
pruned_keras_file = path + model_name + '/pruned_'+ model_name + '.h5'
print('Saving pruned model to: ', pruned_keras_file)

# No need to save the optimizer with the graph for serving.
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

Saving pruned model to:  drive/My Drive/interns/Kpit/alexnet/pruned_alexnet.h5


In [0]:
zip1 = path + model_name + '/raw_'+ model_name + '.zip' 
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("Size of the unpruned model before compression: %.2f Mb" % 
      (os.path.getsize(keras_file) / float(2**20)))
print("Size of the unpruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip1) / float(2**20)))

zip2 = path + model_name + '/pruned_'+ model_name + '.zip' 
with zipfile.ZipFile(zip2, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" % 
      (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip2) / float(2**20)))

Size of the unpruned model before compression: 7.28 Mb
Size of the unpruned model after compression: 6.69 Mb
Size of the pruned model before compression: 7.28 Mb
Size of the pruned model after compression: 1.44 Mb


In [0]:
tflite_model_file = path + model_name + '/pruned_'+ model_name + '.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

W0711 05:42:40.410294 140621474404224 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [0]:
zip_tflite = path + model_name + '/pruned_'+ model_name + '_tflite.zip'
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 7.18 Mb
Size of the tflite model after compression: 1.40 Mb


In [0]:
import numpy as np

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 48, 48, 1))
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))

  return float(num_correct) / float(total_seen)


print(eval_model(interpreter, x_test, y_test))

Accuracy after 1000 images: 0.608000
Accuracy after 2000 images: 0.624000
Accuracy after 3000 images: 0.618333
0.6168849261632767


In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = path + model_name + 'pruned_'+ model_name + '_quant.tflite'
with open(tflite_quant_model_file, 'wb') as f:
  f.write(tflite_quant_model)

W0711 05:43:03.651521 140621474404224 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [0]:
zip_tflite = path + model_name + '/pruned_quant_'+ model_name + '_tflite.zip'
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_quant_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 1.81 Mb
Size of the tflite model after compression: 0.34 Mb


In [0]:
interpreter = tf.lite.Interpreter(model_path=path + model_name + 'pruned_'+ model_name + '_quant.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

num_correct = 0
aa = time.time()
for img,label in zip(x_test, y_test):
    inp = img.reshape((1,48,48,1))
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    if np.argmax(output) == np.argmax(label):
        num_correct = num_correct + 1
time_elapsed = time.time() - aa

print('total_images : ', len(x_test), 'accuracy : ' , float(num_correct)/len(x_test))
print('total time :', time_elapsed , 'time per image: ', time_elapsed/len(x_test), 'fps : ', len(x_test)/time_elapsed)


total_images :  3589 accuracy :  0.6179994427417108
total time : 29.06096625328064 time per image:  0.008097232168648827 fps :  123.49899066397506
